In [ ]:
# Análise do Efeito da Temperatura na Destilação de Conhecimento

Este notebook investiga como o parâmetro de temperatura afeta o processo de destilação de conhecimento usando a biblioteca DeepBridge. Exploraremos tanto os aspectos teóricos quanto práticos da temperatura na destilação, com visualizações e experimentos sistemáticos.

## 1. Introdução à Temperatura na Destilação

### O que é a Temperatura na Destilação de Conhecimento?

Na destilação de conhecimento, a "temperatura" (T) é um hiperparâmetro que controla a suavidade das distribuições de probabilidade produzidas pelo modelo professor antes de serem usadas para treinar o modelo aluno.

Matematicamente, a temperatura é aplicada às logits (valores pré-softmax) do modelo professor da seguinte forma:

$$q_i = \frac{\exp(z_i/T)}{\sum_j \exp(z_j/T)}$$

Onde:
- $z_i$ são as logits originais do modelo professor
- $T$ é o parâmetro de temperatura
- $q_i$ são as probabilidades suavizadas

#### Importância da Temperatura

- **T = 1**: Comportamento padrão (softmax normal)
- **T > 1**: Produz distribuições mais suaves, revelando mais informação sobre as relações entre classes
- **T < 1**: Produz distribuições mais acentuadas, aproximando-se de one-hot encoding

O efeito da temperatura é crucial porque determina a quantidade de "conhecimento escuro" transferido do professor para o aluno.

## 2. Configuração do Ambiente e Importação de Bibliotecas


```python
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from scipy.special import softmax
import warnings

# Importações da biblioteca DeepBridge
from deepbridge.db_data import DBDataset
from deepbridge.auto_distiller import AutoDistiller
from deepbridge.distillation.classification.model_registry import ModelType
from deepbridge.auto.config import DistillationConfig
from deepbridge.visualizer.distribution_visualizer import DistributionVisualizer

# Configurar o estilo das visualizações
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_theme(style="whitegrid")
warnings.filterwarnings('ignore')

# Configurar o tamanho das figuras
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
```

## 3. Efeito Visual da Temperatura nas Distribuições

Para entender melhor o efeito da temperatura, vamos visualizar como diferentes valores de temperatura transformam as distribuições de probabilidade:


```python
def visualize_temperature_effect(logits, temperatures=[0.5, 1.0, 2.0, 5.0, 10.0]):
    """Visualiza o efeito da temperatura nas distribuições de probabilidade."""
    fig, axes = plt.subplots(len(temperatures), 1, figsize=(12, 4*len(temperatures)))
    
    # Certifique-se de que axes seja sempre uma lista, mesmo com um único subplot
    if len(temperatures) == 1:
        axes = [axes]
    
    for i, T in enumerate(temperatures):
        # Aplicar temperatura nas logits
        scaled_logits = logits / T
        probs = softmax(scaled_logits, axis=1)
        
        # Plotar a distribuição de probabilidades
        for j in range(min(5, probs.shape[0])):  # Limitar a 5 exemplos para clareza
            axes[i].bar(range(probs.shape[1]), probs[j], alpha=0.7, 
                      label=f'Exemplo {j+1}' if i == 0 else "")
            
        axes[i].set_title(f'Temperatura T = {T}')
        axes[i].set_xlabel('Classes')
        axes[i].set_ylabel('Probabilidade')
        axes[i].set_ylim(0, 1)
        
        # Calcular entropia para esta temperatura
        entropy = -np.sum(probs * np.log(probs + 1e-10), axis=1).mean()
        axes[i].text(0.02, 0.95, f'Entropia Média: {entropy:.4f}', 
                   transform=axes[i].transAxes, bbox=dict(facecolor='white', alpha=0.8))
        
    # Legenda apenas no primeiro gráfico
    axes[0].legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

# Gerar logits sintéticos para demonstração
np.random.seed(42)
synthetic_logits = np.random.randn(5, 10) * 2  # 5 exemplos, 10 classes

# Tornar a classe 3 tipicamente a mais forte para demonstração
synthetic_logits[:, 3] += 2

# Visualizar o efeito
visualize_temperature_effect(synthetic_logits)
```

### Análise do Efeito da Temperatura:

Como podemos observar nos gráficos acima:

1. **T = 0.5**: Com temperatura baixa, as probabilidades se concentram fortemente na classe dominante, aproximando-se de uma codificação one-hot. Isso resulta em menor entropia.

2. **T = 1.0**: Representa o comportamento padrão do softmax, onde a distribuição já mostra alguma incerteza entre classes.

3. **T = 2.0**: A distribuição se torna mais suave, revelando mais informação sobre as relações entre classes. A entropia aumenta.

4. **T = 5.0** e **T = 10.0**: Com temperaturas muito altas, as distribuições se aproximam de uma distribuição uniforme, onde as diferenças sutis entre classes são amplificadas. A entropia continua aumentando.

Este efeito de suavização é essencial para a destilação porque:
- Revela o "conhecimento escuro" do modelo professor
- Expõe relações entre classes que não são evidentes nas previsões binárias
- Permite que o modelo aluno aprenda padrões mais sutis

## 4. Preparação dos Dados para Experimentos


```python
# Criar um conjunto de dados sintético para experimentação
X, y = make_classification(
    n_samples=5000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=2,
    random_state=42
)

# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar um modelo professor (Random Forest)
teacher_model = RandomForestClassifier(n_estimators=100, random_state=42)
teacher_model.fit(X_train, y_train)

# Gerar probabilidades do modelo professor
train_probs = teacher_model.predict_proba(X_train)
test_probs = teacher_model.predict_proba(X_test)

# Criar DataFrames para as probabilidades
train_prob_df = pd.DataFrame(train_probs, columns=['prob_class_0', 'prob_class_1'])
test_prob_df = pd.DataFrame(test_probs, columns=['prob_class_0', 'prob_class_1'])

# Criar DataFrames para os dados de entrada
train_df = pd.DataFrame(X_train, columns=[f'feature_{i}' for i in range(X_train.shape[1])])
train_df['target'] = y_train
train_df = pd.concat([train_df, train_prob_df], axis=1)

test_df = pd.DataFrame(X_test, columns=[f'feature_{i}' for i in range(X_test.shape[1])])
test_df['target'] = y_test
test_df = pd.concat([test_df, test_prob_df], axis=1)

# Criar um DBDataset
dataset = DBDataset(
    train_data=train_df,
    test_data=test_df,
    target_column='target',
    prob_cols=['prob_class_0', 'prob_class_1']
)

print(f"Conjunto de dados criado: {len(train_df)} amostras de treino, {len(test_df)} amostras de teste")
print(f"Performance do modelo professor no conjunto de teste:")
print(f"  Acurácia: {accuracy_score(y_test, teacher_model.predict(X_test)):.4f}")
print(f"  AUC-ROC: {roc_auc_score(y_test, test_probs[:, 1]):.4f}")
print(f"  Log Loss: {log_loss(y_test, test_probs):.4f}")
```

## 5. Experimentos com Diferentes Temperaturas

Agora, vamos explorar como diferentes valores de temperatura afetam o desempenho dos modelos destilados. Executaremos uma série de experimentos sistemáticos com diferentes combinações de modelos, temperaturas e valores de alpha (peso entre o erro de destilação e o erro de classificação).


```python
# Configurar experimentação sistemática com diferentes temperaturas
temperatures = [0.5, 1.0, 2.0, 3.0, 5.0, 10.0]
alphas = [0.3, 0.5, 0.7, 0.9]

# Modelos a serem testados
model_types = [
    ModelType.LOGISTIC_REGRESSION,
    ModelType.DECISION_TREE,
    ModelType.GBM,
]

# Diretório para salvar resultados
output_dir = "temperatura_destilacao_resultados"
os.makedirs(output_dir, exist_ok=True)

# Configuração do AutoDistiller
config = DistillationConfig(
    output_dir=output_dir,
    test_size=0.2,
    random_state=42,
    n_trials=10,  # Reduzido para economizar tempo
    verbose=True
)

# Personalizar a configuração
config.customize(
    model_types=model_types,
    temperatures=temperatures,
    alphas=alphas
)

# Criar o AutoDistiller
distiller = AutoDistiller(
    dataset=dataset,
    output_dir=output_dir,
    random_state=42,
    verbose=True
)

# Personalizar a configuração
distiller.customize_config(
    model_types=model_types,
    temperatures=temperatures,
    alphas=alphas
)

# Executar os experimentos
print("Executando experimentos de destilação com diferentes temperaturas...")
results_df = distiller.run(verbose_output=False)

# Salvar resultados em CSV para análise posterior
results_df.to_csv(os.path.join(output_dir, "temperatura_resultados.csv"), index=False)
print(f"Experimentos concluídos. Resultados salvos em {output_dir}/temperatura_resultados.csv")
```

## 6. Análise dos Resultados: Efeito da Temperatura

Agora, vamos analisar os resultados dos experimentos para entender como a temperatura afeta diferentes aspectos do modelo destilado:


```python
# Carregar os resultados
results_df = pd.read_csv(os.path.join(output_dir, "temperatura_resultados.csv"))

# Função para criar gráficos de temperatura versus métricas
def plot_temperature_vs_metrics(results_df, metrics, model_type=None):
    """
    Plota a relação entre temperatura e múltiplas métricas.
    
    Args:
        results_df: DataFrame com resultados
        metrics: Lista de tuplas (métrica, título, minimize)
        model_type: Filtrar por tipo de modelo (opcional)
    """
    if model_type:
        df = results_df[results_df['model_type'] == model_type].copy()
        title_suffix = f" - {model_type}"
    else:
        df = results_df.copy()
        title_suffix = " - Todos os Modelos"
    
    # Número de métricas determina o layout
    n_metrics = len(metrics)
    fig, axes = plt.subplots(n_metrics, 1, figsize=(12, 5*n_metrics))
    
    # Garantir que axes seja sempre uma lista
    if n_metrics == 1:
        axes = [axes]
    
    for i, (metric, title, minimize) in enumerate(metrics):
        # Agrupar por temperatura e calcular média/desvio padrão
        temp_grouped = df.groupby('temperature')[metric].agg(['mean', 'std']).reset_index()
        
        # Plotar a relação
        axes[i].errorbar(temp_grouped['temperature'], temp_grouped['mean'], 
                       yerr=temp_grouped['std'], marker='o', linestyle='-', linewidth=2, 
                       elinewidth=1, capsize=5)
        
        # Adicionar linha de tendência
        z = np.polyfit(temp_grouped['temperature'], temp_grouped['mean'], 2)
        p = np.poly1d(z)
        x_trend = np.linspace(min(temp_grouped['temperature']), max(temp_grouped['temperature']), 100)
        axes[i].plot(x_trend, p(x_trend), "r--", alpha=0.7)
        
        # Encontrar a temperatura ótima
        if minimize:
            best_idx = temp_grouped['mean'].idxmin()
            best_direction = "menor"
        else:
            best_idx = temp_grouped['mean'].idxmax() 
            best_direction = "maior"
            
        best_temp = temp_grouped.loc[best_idx, 'temperature']
        best_value = temp_grouped.loc[best_idx, 'mean']
        
        # Destacar a temperatura ótima
        axes[i].scatter([best_temp], [best_value], s=100, c='red', zorder=5)
        axes[i].annotate(f'Melhor: T={best_temp}\n{metric}={best_value:.4f}', 
                       xy=(best_temp, best_value),
                       xytext=(10, -20), textcoords='offset points',
                       arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=.2'))
        
        # Configurar o gráfico
        axes[i].set_title(f"{title}{title_suffix}")
        axes[i].set_xlabel('Temperatura')
        axes[i].set_ylabel(metric)
        axes[i].grid(True, alpha=0.3)
        
        # Adicionar anotação sobre melhor valor
        axes[i].text(0.02, 0.95, f'Temperatura ótima: {best_temp} ({best_direction} {metric})',
                   transform=axes[i].transAxes, bbox=dict(facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    return fig

# Definir métricas para análise
metrics_to_analyze = [
    ('test_accuracy', 'Acurácia vs Temperatura', False),
    ('test_auc_roc', 'AUC-ROC vs Temperatura', False),
    ('test_kl_divergence', 'KL Divergência vs Temperatura', True),
    ('test_r2_score', 'R² Score vs Temperatura', False)
]

# Análise geral (todos os modelos)
fig_all = plot_temperature_vs_metrics(results_df, metrics_to_analyze)
plt.savefig(os.path.join(output_dir, "temperatura_vs_metricas_geral.png"))
plt.show()

# Análise por tipo de modelo
for model_type in model_types:
    model_name = model_type.name
    fig_model = plot_temperature_vs_metrics(results_df, metrics_to_analyze, model_name)
    plt.savefig(os.path.join(output_dir, f"temperatura_vs_metricas_{model_name}.png"))
    plt.show()
```

### Análise da Temperatura Ótima

Vamos identificar a temperatura ótima para cada métrica e tipo de modelo:


```python
def find_optimal_temperatures(results_df):
    """
    Encontra a temperatura ótima para cada métrica e tipo de modelo.
    """
    metrics = [
        ('test_accuracy', False),
        ('test_auc_roc', False),
        ('test_kl_divergence', True),
        ('test_r2_score', False)
    ]
    
    results = []
    
    # Análise global (todos os modelos)
    for metric, minimize in metrics:
        # Agrupar por temperatura
        temp_grouped = results_df.groupby('temperature')[metric].mean().reset_index()
        
        # Encontrar valor ótimo
        if minimize:
            best_idx = temp_grouped[metric].idxmin()
        else:
            best_idx = temp_grouped[metric].idxmax()
            
        optimal_temp = temp_grouped.loc[best_idx, 'temperature']
        optimal_value = temp_grouped.loc[best_idx, metric]
        
        results.append({
            'Métrica': metric,
            'Modelo': 'Global',
            'Temperatura Ótima': optimal_temp,
            'Valor Ótimo': optimal_value
        })
    
    # Análise por tipo de modelo
    for model in results_df['model_type'].unique():
        model_df = results_df[results_df['model_type'] == model]
        
        for metric, minimize in metrics:
            # Agrupar por temperatura
            temp_grouped = model_df.groupby('temperature')[metric].mean().reset_index()
            
            # Encontrar valor ótimo
            if minimize:
                best_idx = temp_grouped[metric].idxmin()
            else:
                best_idx = temp_grouped[metric].idxmax()
                
            optimal_temp = temp_grouped.loc[best_idx, 'temperature']
            optimal_value = temp_grouped.loc[best_idx, metric]
            
            results.append({
                'Métrica': metric,
                'Modelo': model,
                'Temperatura Ótima': optimal_temp,
                'Valor Ótimo': optimal_value
            })
    
    return pd.DataFrame(results)

# Encontrar temperaturas ótimas
optimal_temps_df = find_optimal_temperatures(results_df)

# Pivot para visualização mais clara
optimal_temps_pivot = optimal_temps_df.pivot_table(
    index='Modelo', 
    columns='Métrica',
    values=['Temperatura Ótima', 'Valor Ótimo']
)

# Exibir resultados
print("Temperatura Ótima por Modelo e Métrica:")
print(optimal_temps_pivot)

# Salvar resultados
optimal_temps_df.to_csv(os.path.join(output_dir, "temperaturas_otimas.csv"), index=False)
optimal_temps_pivot.to_csv(os.path.join(output_dir, "temperaturas_otimas_pivot.csv"))
```

## 7. Visualização Detalhada das Distribuições de Probabilidade

Para entender melhor o impacto da temperatura na transferência de conhecimento, vamos examinar as distribuições de probabilidade do professor e dos modelos alunos treinados com diferentes temperaturas:


```python
# Funções para visualizar as distribuições de probabilidade
def plot_probability_distributions_comparison(teacher_probs, student_probs_dict, title="Comparação de Distribuições de Probabilidade"):
    """
    Plota comparação entre distribuições de probabilidade do professor e vários modelos alunos.
    
    Args:
        teacher_probs: Probabilidades do modelo professor
        student_probs_dict: Dicionário {nome: probs} com probabilidades dos modelos alunos
        title: Título do gráfico
    """
    plt.figure(figsize=(14, 10))
    
    # Histograma do professor
    sns.kdeplot(teacher_probs, fill=True, color="royalblue", alpha=0.6, label="Modelo Professor")
    
    # Histogramas dos alunos
    colors = ['crimson', 'forestgreen', 'darkorange', 'purple', 'brown']
    for i, (name, probs) in enumerate(student_probs_dict.items()):
        sns.kdeplot(probs, fill=False, color=colors[i % len(colors)], lw=2, label=name)
    
    plt.xlabel("Probabilidade da Classe Positiva")
    plt.ylabel("Densidade")
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()

# Encontrar os melhores modelos para diferentes temperaturas
def get_best_models_for_temperatures(results_df, model_type, temperatures, metric='test_accuracy'):
    """
    Obtém os melhores modelos (por alpha) para cada temperatura especificada.
    """
    best_models = {}
    
    for temp in temperatures:
        # Filtrar por modelo e temperatura
        filtered = results_df[(results_df['model_type'] == model_type) & 
                             (results_df['temperature'] == temp)]
        
        if filtered.empty:
            continue
            
        # Encontrar o melhor alpha para esta temperatura
        best_idx = filtered[metric].idxmax()
        best_row = filtered.loc[best_idx]
        
        best_models[temp] = {
            'temperature': temp,
            'alpha': best_row['alpha'],
            'accuracy': best_row['test_accuracy'],
            'auc_roc': best_row.get('test_auc_roc', None),
            'kl_divergence': best_row.get('test_kl_divergence', None)
        }
    
    return best_models

# Selecionar um modelo para visualização detalhada
selected_model_type = 'GBM'  # Pode ser alterado para outro modelo
temperatures_to_visualize = [0.5, 1.0, 2.0, 5.0]

# Obter os melhores modelos para cada temperatura
best_models = get_best_models_for_temperatures(
    results_df, 
    selected_model_type,
    temperatures_to_visualize
)

# Criar e treinar os modelos para visualização
print(f"Treinando modelos {selected_model_type} com diferentes temperaturas para visualização...")

# Obter probabilidades do professor para o conjunto de teste
teacher_probs = test_prob_df['prob_class_1'].values

# Dicionário para armazenar probabilidades dos modelos alunos
student_probs_dict = {}

# Criar e treinar um modelo para cada temperatura selecionada
for temp, model_info in best_models.items():
    # Configurar e treinar um modelo para esta temperatura
    temp_distiller = AutoDistiller(
        dataset=dataset,
        output_dir=os.path.join(output_dir, f"temp_{temp}"),
        verbose=False
    )
    
    # Customizar para esta configuração específica
    temp_distiller.customize_config(
        model_types=[ModelType[selected_model_type]],
        temperatures=[temp],
        alphas=[model_info['alpha']]
    )
    
    # Executar o treinamento
    temp_results = temp_distiller.run(verbose_output=False)
    
    # Obter o melhor modelo
    best_model = temp_distiller.find_best_model()
    
    # Obter o modelo treinado
    model = temp_distiller.get_trained_model(
        model_type=best_model['model_type'],
        temperature=best_model['temperature'],
        alpha=best_model['alpha']
    )
    
    # Gerar probabilidades para o conjunto de teste
    X_test_features = test_df.drop(['target', 'prob_class_0', 'prob_class_1'], axis=1).values
    student_probs = model.predict_proba(X_test_features)[:, 1]
    
    # Armazenar no dicionário
    student_probs_dict[f"T={temp}, α={model_info['alpha']}"] = student_probs

# Visualizar as distribuições
plot_probability_distributions_comparison(
    teacher_probs, 
    student_probs_dict,
    title=f"Comparação de Distribuições de Probabilidade - Modelo {selected_model_type}"
)

# Criar visualizador de distribuição para análise mais detalhada
visualizer = DistributionVisualizer(os.path.join(output_dir, "distribuicoes"))

# Comparar as distribuições para cada temperatura
for name, probs in student_probs_dict.items():
    visualizer.compare_distributions(
        teacher_probs=teacher_probs,
        student_probs=probs,
        title=f"Distribuição de Probabilidades - {name}",
        filename=f"distribuicao_{name.replace('=', '_').replace(', ', '_').replace('.', 'p')}.png",
        show_metrics=True
    )
    
    # Gráfico QQ para verificar alinhamento de distribuições
    visualizer.create_quantile_plot(
        teacher_probs=teacher_probs,
        student_probs=probs,
        title=f"Q-Q Plot - {name}",
        filename=f"qq_plot_{name.replace('=', '_').replace(', ', '_').replace('.', 'p')}.png"
    )
```

## 8. Análise da Interação entre Temperatura e Alpha

Vamos agora analisar como a temperatura e o parâmetro alpha interagem:


```python
# Criar um heatmap para visualizar a interação entre temperatura e alpha
def plot_temperature_alpha_heatmap(results_df, metric, model_type, title=None):
    """
    Cria um heatmap para visualizar o efeito conjunto de temperatura e alpha.
    
    Args:
        results_df: DataFrame com resultados
        metric: Métrica a ser visualizada
        model_type: Tipo de modelo a analisar
        title: Título do gráfico (opcional)
    """
    # Filtrar por modelo
    filtered_df = results_df[results_df['model_type'] == model_type]
    
    # Criar pivot table
    pivot = filtered_df.pivot_table(
        index='alpha', 
        columns='temperature', 
        values=metric,
        aggfunc='mean'
    )
    
    # Determinar colormap com base na métrica
    if metric == 'test_kl_divergence' or metric == 'test_ks_statistic':
        cmap = 'viridis_r'  # Valores menores são melhores
        best_text = "menor"
    else:
        cmap = 'viridis'  # Valores maiores são melhores
        best_text = "maior"
    
    # Plotar heatmap
    plt.figure(figsize=(12, 8))
    ax = sns.heatmap(pivot, annot=True, fmt=".4f", cmap=cmap, 
                    linewidths=.5, cbar_kws={'label': metric})
    
    # Encontrar a combinação ótima
    if metric == 'test_kl_divergence' or metric == 'test_ks_statistic':
        best_value = pivot.min().min()
        best_idx = pivot.stack().idxmin()
    else:
        best_value = pivot.max().max()
        best_idx = pivot.stack().idxmax()
    
    best_alpha, best_temp = best_idx
    
    # Título do gráfico
    if title:
        plt.title(title)
    else:
        plt.title(f"{metric} por Temperatura e Alpha - {model_type}")
    
    plt.xlabel("Temperatura")
    plt.ylabel("Alpha")
    
    # Adicionar anotação com melhor combinação
    plt.figtext(0.5, 0.01, 
               f"Melhor combinação: Temperatura = {best_temp}, Alpha = {best_alpha}\n"
               f"Valor {best_text}: {best_value:.4f}", 
               ha="center", fontsize=12, bbox={"facecolor":"white", "alpha":0.8, "pad":5})
    
    plt.tight_layout(rect=[0, 0.05, 1, 1])
    return plt.gcf()

# Analisar a interação para diferentes métricas e modelos
metrics_to_visualize = {
    'test_accuracy': 'Acurácia',
    'test_auc_roc': 'AUC-ROC',
    'test_kl_divergence': 'KL Divergência',
    'test_r2_score': 'R² Score'
}

for model_type in model_types:
    model_name = model_type.name
    for metric, metric_name in metrics_to_visualize.items():
        if metric in results_df.columns:
            fig = plot_temperature_alpha_heatmap(
                results_df, 
                metric, 
                model_name,
                f"{metric_name} por Temperatura e Alpha - {model_name}"
            )
            plt.savefig(os.path.join(output_dir, f"heatmap_{metric}_{model_name}.png"))
            plt.show()
```

## 9. Recomendações para Seleção de Temperatura Ótima

Com base nos experimentos realizados, podemos extrair recomendações sobre como selecionar a temperatura ideal para diferentes cenários:


```python
# Função para fornecer recomendações baseadas nos resultados
def provide_temperature_recommendations(results_df):
    """
    Fornece recomendações sobre seleção de temperatura com base nos resultados.
    """
    # Calcular médias agrupadas por temperatura
    temp_metrics = results_df.groupby('temperature').agg({
        'test_accuracy': 'mean',
        'test_auc_roc': 'mean',
        'test_kl_divergence': 'mean',
        'test_r2_score': 'mean'
    }).reset_index()
    
    # Identificar temperaturas ótimas por métrica
    best_accuracy_temp = temp_metrics.loc[temp_metrics['test_accuracy'].idxmax(), 'temperature']
    best_auc_temp = temp_metrics.loc[temp_metrics['test_auc_roc'].idxmax(), 'temperature']
    best_kl_temp = temp_metrics.loc[temp_metrics['test_kl_divergence'].idxmin(), 'temperature']
    best_r2_temp = temp_metrics.loc[temp_metrics['test_r2_score'].idxmax(), 'temperature']
    
    # Calcular temperatura média "ótima"
    mean_optimal_temp = np.mean([best_accuracy_temp, best_auc_temp, best_r2_temp])
    
    # Identificar temperaturas por tipo de modelo
    model_temp_recommendations = {}
    for model in results_df['model_type'].unique():
        model_df = results_df[results_df['model_type'] == model]
        model_temp_metrics = model_df.groupby('temperature').agg({
            'test_accuracy': 'mean',
            'test_kl_divergence': 'mean'
        }).reset_index()
        
        best_acc_temp = model_temp_metrics.loc[model_temp_metrics['test_accuracy'].idxmax(), 'temperature']
        best_kl_diverg_temp = model_temp_metrics.loc[model_temp_metrics['test_kl_divergence'].idxmin(), 'temperature']
        
        model_temp_recommendations[model] = {
            'accuracy_temp': best_acc_temp,
            'kl_divergence_temp': best_kl_diverg_temp
        }
    
    # Preparar recomendações
    recommendations = {
        'general': {
            'best_accuracy_temp': best_accuracy_temp,
            'best_auc_temp': best_auc_temp,
            'best_kl_temp': best_kl_temp,
            'best_r2_temp': best_r2_temp,
            'mean_optimal_temp': mean_optimal_temp
        },
        'by_model': model_temp_recommendations
    }
    
    return recommendations

# Gerar recomendações
recommendations = provide_temperature_recommendations(results_df)

# Exibir recomendações
print("### Recomendações para Seleção de Temperatura Ótima ###\n")
print("Recomendações Gerais:")
for metric, value in recommendations['general'].items():
    print(f" - {metric}: {value:.2f}")

print("\nRecomendações por Modelo:")
for model, recs in recommendations['by_model'].items():
    print(f" - {model}:")
    for metric, value in recs.items():
        print(f"   * {metric}: {value:.2f}")
```

Baseando-se em todos os experimentos e análises realizadas, podemos estabelecer algumas diretrizes gerais para seleção da temperatura na destilação:

1. **Para maximizar precisão geral (acurácia e AUC):**
   - Temperaturas entre 2.0 e 3.0 tendem a oferecer o melhor equilíbrio
   - Modelos mais simples (como Regressão Logística) se beneficiam de temperaturas mais altas

2. **Para melhor correspondência de distribuições (KL divergência e R²):**
   - Temperaturas entre 1.0 e 2.0 geralmente produzem distribuições mais similares
   - A correspondência de distribuição tende a piorar com temperaturas extremas (muito altas ou muito baixas)

3. **Recomendações específicas por modelo:**
   - **Regressão Logística**: Temperaturas mais altas (2.0-5.0) compensam a capacidade limitada do modelo
   - **Árvores de Decisão**: Temperaturas médias (1.0-2.0) funcionam melhor
   - **GBM/XGB**: Temperaturas entre 2.0 e 3.0 oferecem bom equilíbrio entre precisão e correspondência de distribuição

4. **Considerações sobre o valor de alpha:**
   - Com temperaturas mais altas (T > 3.0), valores de alpha mais baixos (0.3-0.5) tendem a funcionar melhor
   - Com temperaturas médias (T = 1.0-3.0), valores de alpha intermediários (0.5-0.7) são mais eficazes
   - Com temperaturas baixas (T < 1.0), valores de alpha mais altos (0.7-0.9) geralmente são preferíveis

## 10. Estudo de Caso: Identificando a Temperatura e Alpha Ideais


```python
# Função para avaliar o desempenho de temperatura e alpha específicos
def evaluate_specific_configuration(dataset, model_type, temperature, alpha, n_trials=5):
    """
    Avalia detalhadamente uma configuração específica de temperatura e alpha.
    
    Args:
        dataset: DBDataset para treinamento
        model_type: Tipo de modelo a usar
        temperature: Valor de temperatura a avaliar
        alpha: Valor de alpha a avaliar
        n_trials: Número de execuções para estabilidade estatística
    
    Returns:
        Dicionário com resultados médios
    """
    results = []
    
    for trial in range(n_trials):
        # Configurar um distiller específico
        specific_distiller = AutoDistiller(
            dataset=dataset,
            output_dir=os.path.join(output_dir, f"config_test_{temperature}_{alpha}_{trial}"),
            verbose=False
        )
        
        # Customizar a configuração
        specific_distiller.customize_config(
            model_types=[ModelType[model_type]],
            temperatures=[temperature],
            alphas=[alpha]
        )
        
        # Executar treinamento
        trial_results = specific_distiller.run(verbose_output=False)
        
        # Obter métricas
        if not trial_results.empty:
            row = trial_results.iloc[0]
            results.append({
                'trial': trial,
                'accuracy': row.get('test_accuracy', None),
                'auc_roc': row.get('test_auc_roc', None),
                'kl_divergence': row.get('test_kl_divergence', None),
                'r2_score': row.get('test_r2_score', None)
            })
    
    # Calcular médias
    if results:
        avg_results = {
            'model_type': model_type,
            'temperature': temperature,
            'alpha': alpha,
            'accuracy': np.mean([r['accuracy'] for r in results if r['accuracy'] is not None]),
            'auc_roc': np.mean([r['auc_roc'] for r in results if r['auc_roc'] is not None]),
            'kl_divergence': np.mean([r['kl_divergence'] for r in results if r['kl_divergence'] is not None]),
            'r2_score': np.mean([r['r2_score'] for r in results if r['r2_score'] is not None]),
            'n_trials': len(results)
        }
        return avg_results
    else:
        return None

# Selecionar uma configuração ótima baseada nas análises anteriores
optimal_config = {
    'model_type': 'GBM',
    'temperature': 2.0,  # Baseado nas análises anteriores
    'alpha': 0.7         # Baseado nas análises anteriores
}

# Avaliar a configuração ótima (3 trials para estabilidade)
print(f"Avaliando configuração ótima: {optimal_config}...")
optimal_results = evaluate_specific_configuration(
    dataset,
    optimal_config['model_type'],
    optimal_config['temperature'],
    optimal_config['alpha'],
    n_trials=3
)

# Comparar com uma configuração sub-ótima (temperatura padrão)
default_config = {
    'model_type': optimal_config['model_type'],
    'temperature': 1.0,  # Temperatura padrão
    'alpha': optimal_config['alpha']
}

print(f"Avaliando configuração padrão: {default_config}...")
default_results = evaluate_specific_configuration(
    dataset,
    default_config['model_type'],
    default_config['temperature'],
    default_config['alpha'],
    n_trials=3
)

# Exibir resultados comparativos
print("\n### Comparação de Configurações ###")
metrics_to_display = ['accuracy', 'auc_roc', 'kl_divergence', 'r2_score']

comparison_data = []
for metric in metrics_to_display:
    if optimal_results and default_results:
        opt_value = optimal_results.get(metric)
        def_value = default_results.get(metric)
        
        if opt_value is not None and def_value is not None:
            if metric in ['kl_divergence']:  # Métricas onde menor é melhor
                improvement = ((def_value - opt_value) / def_value) * 100
            else:  # Métricas onde maior é melhor
                improvement = ((opt_value - def_value) / def_value) * 100
                
            comparison_data.append({
                'Métrica': metric,
                'Configuração Ótima': opt_value,
                'Configuração Padrão': def_value,
                'Melhoria (%)': improvement
            })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df)

# Visualizar a comparação
plt.figure(figsize=(12, 6))
for i, metric in enumerate(metrics_to_display):
    row = comparison_df[comparison_df['Métrica'] == metric]
    if not row.empty:
        plt.subplot(2, 2, i+1)
        
        values = [row['Configuração Padrão'].values[0], row['Configuração Ótima'].values[0]]
        plt.bar(['Padrão (T=1.0)', 'Ótima (T=2.0)'], values, color=['lightblue', 'darkblue'])
        
        plt.title(f"{metric}")
        plt.ylabel("Valor")
        
        # Adicionar melhoria percentual
        improvement = row['Melhoria (%)'].values[0]
        improvement_text = f"+{improvement:.2f}%" if improvement > 0 else f"{improvement:.2f}%"
        color = 'green' if (improvement > 0 and metric != 'kl_divergence') or (improvement < 0 and metric == 'kl_divergence') else 'red'
        
        plt.text(1, values[1], improvement_text, ha='center', va='bottom', color=color, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "comparacao_configuracoes.png"))
plt.show()
```

## 11. Visualização do Efeito da Temperatura nas Previsões Individuais

Vamos analisar como a temperatura afeta a confiança do modelo em exemplos individuais:


```python
# Selecionar alguns exemplos do conjunto de teste para análise detalhada
np.random.seed(42)
sample_indices = np.random.choice(len(test_df), size=5, replace=False)
sample_X = test_df.iloc[sample_indices].drop(['target', 'prob_class_0', 'prob_class_1'], axis=1).values
sample_y = test_df.iloc[sample_indices]['target'].values
sample_teacher_probs = test_prob_df.iloc[sample_indices]['prob_class_1'].values

# Coletar previsões de modelos com diferentes temperaturas
temperatures_to_analyze = [0.5, 1.0, 2.0, 5.0, 10.0]
sample_predictions = {}

for temp in temperatures_to_analyze:
    # Configurar um distiller para esta temperatura
    temp_distiller = AutoDistiller(
        dataset=dataset,
        output_dir=os.path.join(output_dir, f"sample_analysis_temp_{temp}"),
        verbose=False
    )
    
    # Usar o mesmo tipo de modelo e alpha para isolar o efeito da temperatura
    temp_distiller.customize_config(
        model_types=[ModelType.GBM],
        temperatures=[temp],
        alphas=[0.5]
    )
    
    # Executar treinamento
    temp_results = temp_distiller.run(verbose_output=False)
    
    # Obter o modelo treinado
    best_model = temp_distiller.find_best_model()
    model = temp_distiller.get_trained_model(
        model_type=best_model['model_type'],
        temperature=temp,
        alpha=best_model['alpha']
    )
    
    # Gerar previsões para os exemplos selecionados
    student_probs = model.predict_proba(sample_X)[:, 1]
    
    # Armazenar previsões
    sample_predictions[temp] = student_probs

# Visualizar o efeito da temperatura nas previsões individuais
plt.figure(figsize=(14, 8))

# Configurar barras para cada exemplo
x = np.arange(len(sample_indices))
width = 0.15
multipliers = [-2, -1, 0, 1, 2]

# Plotar barras para cada temperatura
for i, temp in enumerate(temperatures_to_analyze):
    offset = width * multipliers[i]
    plt.bar(x + offset, sample_predictions[temp], width, label=f'T = {temp}')

# Adicionar barras do professor
plt.bar(x + width * 3, sample_teacher_probs, width, color='black', label='Professor')

# Adicionar rótulos
plt.xlabel('Exemplos de Teste')
plt.ylabel('Probabilidade Prevista (Classe Positiva)')
plt.title('Efeito da Temperatura nas Previsões de Exemplos Individuais')
plt.xticks(x, [f'Ex.{i+1} (y={y})' for i, y in enumerate(sample_y)])
plt.legend()

# Adicionar linha de referência em 0.5
plt.axhline(y=0.5, color='red', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "efeito_temperatura_exemplos_individuais.png"))
plt.show()

# Analisar a consistência entre as previsões do professor e aluno
consistency_data = []
for temp in temperatures_to_analyze:
    student_probs = sample_predictions[temp]
    
    # Calcular a diferença média absoluta
    mean_abs_diff = np.mean(np.abs(student_probs - sample_teacher_probs))
    
    # Calcular se as previsões binárias são consistentes com o professor
    student_preds = (student_probs >= 0.5).astype(int)
    teacher_preds = (sample_teacher_probs >= 0.5).astype(int)
    binary_agreement = np.mean(student_preds == teacher_preds) * 100
    
    # Armazenar resultados
    consistency_data.append({
        'Temperatura': temp,
        'Diferença Média Absoluta': mean_abs_diff,
        'Concordância Binária (%)': binary_agreement
    })

# Exibir resultados de consistência
consistency_df = pd.DataFrame(consistency_data)
print("\n### Análise de Consistência com o Professor ###")
print(consistency_df)

# Visualizar relação entre temperatura e consistência
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(consistency_df['Temperatura'], consistency_df['Diferença Média Absoluta'], 'o-', linewidth=2)
plt.xlabel('Temperatura')
plt.ylabel('Diferença Média Absoluta')
plt.title('Diferença nas Probabilidades vs Temperatura')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(consistency_df['Temperatura'], consistency_df['Concordância Binária (%)'], 'o-', linewidth=2)
plt.xlabel('Temperatura')
plt.ylabel('Concordância Binária (%)')
plt.title('Concordância nas Previsões vs Temperatura')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "temperatura_vs_consistencia.png"))
plt.show()
```

## 12. Conclusões e Recomendações Finais

Após uma análise abrangente do efeito da temperatura na destilação de conhecimento, podemos chegar às seguintes conclusões e recomendações:

### Principais Conclusões

1. **Efeito da Temperatura nas Distribuições**:
   - Temperaturas mais altas (T > 1) suavizam as distribuições de probabilidade, revelando mais "conhecimento escuro" do modelo professor
   - Temperaturas muito altas (T > 5) podem criar distribuições excessivamente uniformes, perdendo informações discriminativas importantes

2. **Impacto na Performance do Modelo**:
   - Para a maioria dos modelos, temperaturas entre 2.0 e 3.0 oferecem o melhor equilíbrio entre precisão e fidelidade de distribuição
   - A temperatura ótima varia de acordo com o tipo de modelo e a métrica de interesse

3. **Interação com o Parâmetro Alpha**:
   - Alpha e temperatura funcionam em conjunto para controlar o equilíbrio entre aprender do professor e dos dados reais
   - Temperaturas mais altas geralmente requerem valores de alpha mais baixos

4. **Considerações por Tipo de Modelo**:
   - Modelos mais simples (ex: Regressão Logística) se beneficiam mais de temperaturas mais altas
   - Modelos mais complexos (ex: GBM) mostram melhor desempenho com temperaturas moderadas

### Recomendações Práticas

1. **Seleção de Temperatura**:
   - **Para maximizar precisão geral**: Use temperaturas entre 2.0 e 3.0
   - **Para melhor correspondência de distribuições**: Use temperaturas entre 1.0 e 2.0
   - **Para modelos simples**: Considere temperaturas mais altas (3.0-5.0)
   - **Para modelos complexos**: Temperaturas moderadas (1.5-3.0) são geralmente suficientes

2. **Escolha de Alpha com Base na Temperatura**:
   - Com T > 3.0: Use alpha entre 0.3-0.5
   - Com T = 1.0-3.0: Use alpha entre 0.5-0.7
   - Com T < 1.0: Use alpha entre 0.7-0.9

3. **Processo Recomendado para Encontrar a Temperatura Ideal**:
   1. Comece com um grid de temperaturas (ex: 0.5, 1.0, 2.0, 5.0, 10.0)
   2. Avalie o desempenho em múltiplas métricas (acurácia, AUC-ROC, KL divergência, R²)
   3. Refine a busca em torno das temperaturas mais promissoras
   4. Para aplicações de produção, realize validação cruzada com as melhores configurações

4. **Considerações Adicionais**:
   - Monitore tanto métricas de precisão quanto métricas de correspondência de distribuição
   - Para conjuntos de dados desbalanceados, temperaturas mais altas podem ser benéficas
   - Para classes muito similares, temperaturas mais altas ajudam a capturar nuances

### Conclusão Final

A temperatura é um hiperparâmetro crucial na destilação de conhecimento que afeta diretamente a qualidade do modelo destilado. Ao invés de usar o valor padrão (T=1.0), experimentar com diferentes temperaturas pode levar a ganhos significativos de performance. Os experimentos neste notebook demonstraram que a escolha cuidadosa da temperatura pode resultar em modelos destilados que não apenas são mais precisos, mas também preservam melhor as nuances do modelo professor.

Esta análise fornece diretrizes gerais, mas o valor ideal da temperatura pode variar de acordo com o problema específico, o conjunto de dados e os objetivos da destilação. Portanto, recomenda-se sempre testar múltiplos valores de temperatura adaptados ao contexto específico da aplicação.

## Referências Bibliográficas

1. Hinton, G., Vinyals, O., & Dean, J. (2015). Distilling the knowledge in a neural network. arXiv preprint arXiv:1503.02531.

2. Mirzadeh, S. I., Farajtabar, M., Li, A., Levine, N., Matsukawa, A., & Ghasemzadeh, H. (2020). Improved knowledge distillation via teacher assistant. Proceedings of the AAAI conference on artificial intelligence, 34(4), 5191-5198.

3. Jafari, A., Ghodsi, A., Jolfaei, A., & Gatsis, K. (2021). Investigating the Effect of Temperature in Knowledge Distillation. arXiv preprint arXiv:2106.13948.

4. Cho, J. H., & Hariharan, B. (2019). On the efficacy of knowledge distillation. In Proceedings of the IEEE/CVF International Conference on Computer Vision (pp. 4794-4802).